In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Обработка данных

In [2]:
#загружаем основной тренировочный датасет
X_data = pd.read_csv('X_data.csv', sep = ';')

In [3]:
#переименовываем столбец
X_data = X_data.rename(columns = {'Unnamed: 0':'Date'})

In [4]:
#загружаем дополнительные данные для тренировки, где есть целевой показатель MAE
Y_data = pd.read_csv('Y_train.csv', sep = ';')

In [5]:
#исправляем ситуацию с данными в хэдере, корректно называем столбцы
Y_data.loc[-1] = [pd.Series(Y_data.columns)[0],pd.Series(Y_data.columns)[1]]
Y_data.index = Y_data.index + 1
Y_data = Y_data.sort_index()
Y_data = Y_data.rename(columns={'2015-01-04 00:05:00':'Date','392':'MAE'})

In [6]:
#мерджим таблицы для получения финального датасета для тренировки модели
x_data = pd.merge(X_data, Y_data, how = 'right', on = 'Date')

In [7]:
#загружаем датасет с данными, которые следует предсказать
Y_submit = pd.read_csv('Y_submit.csv', sep = ';')

In [8]:
#снова исправляем ситуацию с данными в хэдере, корректно называем столбцы
Y_submit.loc[-1] = [pd.Series(Y_submit.columns)[0],pd.Series(Y_submit.columns)[1]]
Y_submit.index = Y_submit.index + 1
Y_submit = Y_submit.sort_index()
Y_submit = Y_submit.rename(columns={'2018-05-04 00:05:00':'Date','420':'MAE'})

In [9]:
#Мерджим таблицы для получения тестового датасета на котором будут предсказаны неизвестные значения MAE 
X_test = pd.merge(X_data, Y_submit, how = 'right', on = 'Date').drop(['MAE'],axis=1)

In [10]:
#делим тренировочный датасет на тренировочную и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(x_data, 
                                                      x_data['MAE'],
                                                      test_size = 0.3, 
                                                      random_state = 42)

### Моделирование

In [11]:
X_train.columns

Index(['Date', 'T_data_1_1', 'T_data_1_2', 'T_data_1_3', 'T_data_2_1',
       'T_data_2_2', 'T_data_2_3', 'T_data_3_1', 'T_data_3_2', 'T_data_3_3',
       'T_data_4_1', 'T_data_4_2', 'T_data_4_3', 'T_data_5_1', 'T_data_5_2',
       'T_data_5_3', 'H_data', 'AH_data', 'MAE'],
      dtype='object')

In [12]:
#определяем признаки для использования, предполагаю использовать все столбцы, кроме Даты и, естественно, MAE
feats = ['T_data_1_1', 'T_data_1_2', 'T_data_1_3', 'T_data_2_1',
       'T_data_2_2', 'T_data_2_3', 'T_data_3_1', 'T_data_3_2', 'T_data_3_3',
       'T_data_4_1', 'T_data_4_2', 'T_data_4_3', 'T_data_5_1', 'T_data_5_2',
       'T_data_5_3', 'H_data', 'AH_data']

In [13]:
#загружаем метрики и модель вместе с инструментом кросс-валидации
from sklearn.metrics import r2_score as r2
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

### Данный раздел исключен, т.к. занимает более 30 минут на выполнение. 
### Лучшие параметры для модели (11,11,200) были занесены сразу в функцию.

#### настраиваем параметры
parameters = [{'n_estimators': [50, 100, 200], 
               'max_features': np.arange(4, 12), 
               'max_depth': np.arange(7, 12)}]

##### проводим кросс-валидацию
CV = GridSearchCV(estimator = RandomForestRegressor(random_state = 42),
                   param_grid = parameters,
                   scoring = 'r2',
                   cv = 5)

CV.fit(X_train.loc[: , feats], y_train)

#### Определяем наилучшие параметры для RandomForestRegressor
best_params = CV.best_params_

    model = RandomForestRegressor(max_depth = best_params['max_depth'], 
                                  max_features = best_params['max_features'], 
                                  n_estimators = best_params['n_estimators'], 
                                  random_state = 42)

In [14]:
#Создаем функцию, строим модель на лучших параметрах, обучаем её, получаем предсказания и готовую модель на выходе
def model_predict (X_t, X_v, y_t):
    model = RandomForestRegressor(max_depth = 11,
                                  max_features = 11, 
                                  n_estimators = 200, 
                                  random_state = 42)
    model.fit(X_t.loc[: , feats], y_t)
    y_pred_t = model.predict(X_t.loc[: , feats])
    y_pred_v = model.predict(X_v.loc[: , feats])
    return y_pred_t, y_pred_v, model

In [15]:
y_pred_train, y_pred_valid,  model  = model_predict(X_train, X_valid, y_train)

In [16]:
#получаем результат на тренировочном предсказании
r2(y_train, y_pred_train)

0.9645350411256641

In [17]:
#получаем результат на валидационном предсказании
r2(y_valid, y_pred_valid)

0.9115296841678389

In [18]:
#определяем наиболее важные признаки
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = feats,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
T_data_3_1,0.349099
T_data_3_2,0.256490
T_data_3_3,0.225276
H_data,0.030671
T_data_5_1,0.025478
T_data_5_2,0.020336
T_data_5_3,0.018202
T_data_1_1,0.015853
T_data_1_2,0.014317
T_data_1_3,0.013447


In [19]:
#предсказываем результаты на тестовой выборке
X_test['MAE'] = model.predict(X_test.loc[: , feats])
X_test.loc[:,['Date','MAE']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5808 entries, 0 to 5807
Data columns (total 2 columns):
Date    5808 non-null object
MAE     5808 non-null float64
dtypes: float64(1), object(1)
memory usage: 136.1+ KB


In [20]:
#выгружаем результаты
#X_test.loc[:,['Date','MAE']].to_csv('NSMaslov_predictions.csv', index = None)